In [6]:
import pandas as pd
import networkx as nx

Reading data

In [7]:
df=pd.read_json('contacts.json')
df.Email=df.Email.apply(lambda x: "Email_"+x if x !='' else '')
df.Phone=df.Phone.apply(lambda x: "Phone_"+x if x !='' else '')
df.OrderId=df.OrderId.apply(lambda x: "OrderId_"+x if x !='' else '')

Creating Graph G

In [8]:
#Create graph G with edges id <-> email, id <-> phone, id <-> orderId
nodes=[]

for _,Id,_,_,Contacts,_ in df.itertuples():
    nodes.append((Id,{"Contacts": Contacts}))
    
G=nx.Graph()
G.add_nodes_from(nodes)

G.add_edges_from(df[df.Email != ''][['Id', 'Email']].to_records(index=False))
G.add_edges_from(df[df.Phone != ''][['Id', 'Phone']].to_records(index=False))
G.add_edges_from(df[df.OrderId != ''][['Id', 'OrderId']].to_records(index=False))

print(G.number_of_nodes(), G.number_of_edges())

1129135 837231


Finding connected components in Graph G

In [9]:
#number of connected components
conn_comp=list(nx.connected_components(G))

Analysis

In [48]:
users=[]
contact_sum=[]
unique_orders=[]
# for each connected component we apended the ticket_id within

for each_connected_component in conn_comp:
    
    id_list=[]
    order_list=[]

    for each_node in each_connected_component:
        
        # check if the node is a number, append to the id_list
        if str(each_node).isnumeric():
            id_list.append(each_node)

        #check if the node is an OrderId, append to the order_list
        if str(each_node).startswith('OrderId_'):
            order_list.append(each_node)

    # sum of contacts that a node has made
    sum_of_contacts=0
    for order_id in id_list:
        sum_of_contacts += G.nodes[order_id]['Contacts']

    output_str1='-'.join([str(each_node) for each_node in sorted(id_list)])
    
    output_str2=','.join([str(each_node) for each_node in sorted(order_list)])

    for id in id_list:
        users.append([id, output_str1])
        contact_sum.append(sum_of_contacts)
        
    for order_id in id_list:
        unique_orders.append([order_id,output_str2,len(order_list)])

Output 1 - Merging tickets belonging to the same user

In [53]:
output_final1=pd.DataFrame(users)
output_final1=output_final1.rename(columns={0:'id', 1:'ticket_trace'})
output_final1.sort_values('id',inplace=True)
print(output_final1)

            id                              ticket_trace
0            0                                         0
1            1  1-2458-98519-115061-140081-165605-476346
8            2                    2-159312-322639-348955
12           3                                         3
13           4                                         4
...        ...                                       ...
499996  499995                                    499995
499997  499996                                    499996
499998  499997                                    499997
236554  499998                             121111-499998
499999  499999                                    499999

[500000 rows x 2 columns]


Output 2 - Number of times a user has contacted customer service

In [54]:
output_final2=pd.DataFrame(contact_sum)
output_final2=output_final2.rename(columns={0:'No. of contacts made'})
output_final2.sort_values('No. of contacts made',ascending=False,inplace=True)
print(output_final2)

        No. of contacts made
60351                     99
9063                      99
60339                     99
9056                      99
9057                      99
...                      ...
304023                     0
304016                     0
454397                     0
304011                     0
499999                     0

[500000 rows x 1 columns]


Output 3 - Number of unique orders a user has made

In [55]:
output_final3=pd.DataFrame(unique_orders)
output_final3=output_final3.rename(columns={0: 'id', 1:'Unique Orders', 2:'No. of Orders'})
output_final3.sort_values('No. of Orders',ascending=False,inplace=True)
print(output_final3)

            id                                      Unique Orders  \
9072    304811  OrderId_BNiqIFerGaPGDfHYQtEMDctQz,OrderId_BZfV...   
8524    102740  OrderId_ATdeYmjfHGpRkxOEiUQEYIEHz,OrderId_DFKI...   
8519     18722  OrderId_ATdeYmjfHGpRkxOEiUQEYIEHz,OrderId_DFKI...   
8520    443180  OrderId_ATdeYmjfHGpRkxOEiUQEYIEHz,OrderId_DFKI...   
8521     12596  OrderId_ATdeYmjfHGpRkxOEiUQEYIEHz,OrderId_DFKI...   
...        ...                                                ...   
148343   61814                                                      
350579  235657                                                      
350580  235658                                                      
350581  289537                                                      
0            0                                                      

        No. of Orders  
9072               22  
8524               22  
8519               22  
8520               22  
8521               22  
...               ...  
148